协作型过滤：我们知道，要想了解商品、影片或娱乐性网站的推荐信息，最没有技术含量的方法莫过于向朋友们询问。我们也知道，这其中有一部分人的品味会比其他人的高一些，通过观察这些人是否通常也和我们一样喜欢同样的东西，可以逐渐对这些情况有所了解。不过随着选择越来越多，要想通过询问一小群人来确定我们想要的东西，将会变得越来越不切实际，因为他们可能并不了解所有的选择。这就是人们为什么要发展出一套被称为协作型过滤的技术。

一个协作型过滤算法通常的做法是对一大群人进行搜索，并从中找出与我们品味相近的一小群人。算法会对这些人所偏爱的其他内容进行考察，并将它们组合起来构造出一个经过排名的推荐列表，有许多不同的方法可以帮助我们确定哪些人与自己的品味相近，并将他们的选择组合成列表。


In [9]:
from recommendations import critics

In [3]:
critics['Lisa Rose']['Lady in the Water']

2.5

In [4]:
critics['Toby']['Snakes on a Plane'] = 4.5

In [5]:
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

尽管可以将相当数量的人员偏好信息置于字典内（即内存中），但对于一个规模巨大的数据集而言，也许我们还是会希望将其存入数据库中。

寻找相近的用户

计算相似度评价值的方法：欧几里得距离和皮尔逊相关度

欧几里得距离评价

In [6]:
from math import sqrt

In [7]:
sqrt(pow(4.5 - 4, 2) + pow(1 - 2, 2))

1.118033988749895

上述算式可以计算出距离值，偏好越相似的人，其距离就越短。此外，我们还需要一个函数，来对偏好越相近的情况给出越大的值。为此，我们可以将函数值加1（这样可以避免遇到被0整除的错误了），并取其倒数。

In [8]:
1 / (1 + sqrt(pow(4.5 - 4, 2) + pow(1 - 2, 2)))

0.4721359549995794

这一新的函数总是返回介于0到1之间的值，返回1则表示两人具有一样的偏好。

In [27]:
from importlib import reload
reload(recommendations)
from recommendations import sim_distance
sim_distance(critics, 'Lisa Rose', 'Gene Seymour')

0.29429805508554946

上述执行过程给出了Lisa Rose和Gene Seymour之间的相似度评价。请用其他人的名字试试，看看我们是否能够找到或多或少具有一定共性的人。

In [32]:
p=[]
for p1 in critics.keys():
    for p2 in critics.keys():
        if p1 != p2 and p2 not in p:
            print("The distance between " + p1 + ' and ' + p2 +  ' is: ' + str(sim_distance(critics, p1,p2)))
        p.append(p1)

The distance between Jack Matthews and Toby is: 0.2674788903885893
The distance between Jack Matthews and Gene Seymour is: 0.6666666666666666
The distance between Jack Matthews and Lisa Rose is: 0.3405424265831667
The distance between Jack Matthews and Claudia Puig is: 0.32037724101704074
The distance between Jack Matthews and Michael Phillips is: 0.29429805508554946
The distance between Jack Matthews and Mick LaSalle is: 0.2857142857142857
The distance between Toby and Gene Seymour is: 0.25824569976124334
The distance between Toby and Lisa Rose is: 0.3483314773547883
The distance between Toby and Claudia Puig is: 0.3567891723253309
The distance between Toby and Michael Phillips is: 0.38742588672279304
The distance between Toby and Mick LaSalle is: 0.4
The distance between Gene Seymour and Lisa Rose is: 0.29429805508554946
The distance between Gene Seymour and Claudia Puig is: 0.28172904669025317
The distance between Gene Seymour and Michael Phillips is: 0.3090169943749474
The distance

皮尔逊相关度评价

除了欧几里得距离，还有一种更复杂一些的方法可以用来判断人们兴趣的相似度，那就是皮尔逊相关系数。该相关系数是判断两组数据与某一直线拟合程度的一种度量。对应的公式要比欧几里得距离评价的计算公式要复杂，但是它在数据不是很规范（normalized）的时候（比如，影评者对影片的评价总是相对于平均水平偏离很大时），会倾向于给出更好的结果。